In [2]:
import PyPDF2

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp


from langchain.embeddings import HuggingFaceEmbeddings # import hf embedding
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [4]:
pdf_file=["C:/Users/Mrinal Kalita/Python Projects/AIML Capstone Project - CV - Pneumonia Detection-1.pdf"]

In [9]:
for i in pdf_file:
    pdf_reader = PyPDF2.PdfReader(i)
    for index, text in enumerate(pdf_reader.pages):
        print(index,text)

0 {'/Type': '/Page', '/Parent': IndirectObject(2, 0, 2242460633408), '/Resources': {'/ProcSet': ['/PDF', '/Text', '/ImageB', '/ImageC', '/ImageI'], '/ColorSpace': {'/Cs1': IndirectObject(5, 0, 2242460633408), '/Cs2': IndirectObject(7, 0, 2242460633408)}, '/ExtGState': {'/Gs1': IndirectObject(16, 0, 2242460633408), '/Gs2': IndirectObject(17, 0, 2242460633408), '/GS-9742682568': {'/Type': '/ExtGState', '/ca': 0}, '/GS-9750469207': {'/Type': '/ExtGState', '/ca': 0}, '/GS-8450180107': {'/Type': '/ExtGState', '/ca': 1}, '/GS-5824662338': {'/Type': '/ExtGState', '/ca': 1}}, '/Font': {'/Tc1': IndirectObject(8, 0, 2242460633408), '/Tc2': IndirectObject(9, 0, 2242460633408), '/Tc3': IndirectObject(12, 0, 2242460633408), '/Tc4': IndirectObject(13, 0, 2242460633408), '/Helvetica-7098480789': IndirectObject(257, 0, 2242460633408), '/Helvetica-2000805986': IndirectObject(257, 0, 2242460633408), '/Helvetica-7572533686': IndirectObject(257, 0, 2242460633408), '/Helvetica-8659871878': IndirectObject(2